Тема урока: декораторы
Сохранение атрибутов __name__ и __doc__ для декорируемой функции
Применение декоратора functools.wraps
Шаблон декоратора общего назначения
Декоратор, измеряющий время выполнения функции
Декоратор отслеживания количества вызовов функции
Декоратор замедления времени выполнения функции
Декораторы с аргументами
Аннотация. Урок посвящен декораторам.

Сохранение атрибутов __name__ и __doc__ для декорируемой функции

Как мы уже знаем, все функции содержат специальные атрибуты __name__ и __doc__, которые содержат полезную информацию:

__name__ — имя функции
__doc__ — строка документации

In [1]:
from typing import Callable, Any, Tuple, Dict


def greet(name):
    '''Функция приветствия пользователя.'''
    return f'Hello {name}!'

print(greet.__name__)
print(greet.__doc__)

greet
Функция приветствия пользователя.


Рассмотрим применение декоратора bold к функции greet().

In [2]:
def bold(func):
    def wrapper(*args, **kwargs):
        return '<b>' + func(*args, **kwargs) + '</b>'
    return wrapper

@bold
def greet(name):
    '''Функция приветствия пользователя.'''
    return f'Hello {name}!'

print(greet.__name__)
print(greet.__doc__)

wrapper
None


После того как к функции greet() был применен декоратор, её атрибуты __name__ и __doc__ изменились на имя и строку документации внутренней функции wrapper() декоратора bold. Хотя чисто технически это верно, это не очень хорошо.

Одно из решений такой проблемы выглядит следующим образом:

In [3]:
def bold(func):
    def wrapper(*args, **kwargs):
        return '<b>' + func(*args, **kwargs) + '</b>'
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    return wrapper

@bold
def greet(name):
    '''Функция приветствия пользователя.'''
    return f'Hello {name}!'

print(greet.__name__)
print(greet.__doc__)

greet
Функция приветствия пользователя.


Теперь у функции greet() атрибуты __name__ и __doc__ не перетираются после применения декоратора.

Применение декоратора functools.wraps

Решение проблемы связанное с перетиранием атрибутов __name__ и __doc__ является рабочим, однако каждый раз писать в декораторе строки кода, которые восстанавливают значения данных атрибутов, не очень хорошо.

Для решения проблемы перетирания данных атрибутов на практике используют другой декоратор, который находится в модуле functools и называется wraps. Таким образом, чтобы предотвратить перетирание атрибутов __name__ и __doc__ декорируемой функции, декораторы должны использовать декоратор functools.wraps, который сохраняет информацию о первоначальной функции.

In [4]:
import functools

def bold(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        return '<b>' + func(*args, **kwargs) + '</b>'
    return wrapper

@bold
def greet(name):
    '''Функция приветствие пользователя.'''
    return f'Hello {name}!'

print(greet.__name__)
print(greet.__doc__)

greet
Функция приветствие пользователя.


еперь у функции greet() атрибуты __name__ и __doc__ не перетираются после применения декоратора bold.

Шаблон декоратора общего назначения

Все декораторы в большинстве случаев делают примерно одно и то же. Наиболее частый шаблон декоратора выглядит следующим образом:

In [ ]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Что-то выполняется до вызова декорируемой функции
        value = func(*args, **kwargs)
        # декорируется возвращаемое значение функции
        # или что-то выполняется после вызова декорируемой функции
        return value
    return wrapper

 На основе этого шаблона можно строить декораторы общего назначения.

Декоратор измерения времени работы функции

Следующий декоратор измеряет и выводит время выполнения декорируемой функции. Декоратор вычисляет время непосредственно перед запуском функции и сразу после ее завершения и выводит разницу подсчитанных времен.

In [5]:
import functools, time

def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        val = func(*args, **kwargs)
        end = time.perf_counter()
        work_time = end - start
        print(f'Время выполнения {func.__name__}: {round(work_time, 4)} сек.')
        return val
    return wrapper

@timer
def test(n):
    return sum([(i/99)**2 for i in range(n)])

@timer
def sleep(n):
    time.sleep(n)

res1 = test(10000)
res2 = sleep(4)

print(f'Результат функции test = {res1}')
print(f'Результат функции sleep = {res2}')

Время выполнения test: 0.0014 сек.
Время выполнения sleep: 4.0003 сек.
Результат функции test = 34005033.67003367
Результат функции sleep = None


Декоратор отслеживания количества вызовов функции

Иногда полезно иметь декоратор, который может отслеживать состояние вызова функции. Создадим декоратор, который подсчитывает, сколько раз вызывается функция. Для сохранения состояния счетчика будем использовать пользовательский атрибут функции.

In [8]:
import functools

def counter(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.num += 1
        print(f'Вызов {func.__name__}: {wrapper.num}')
        val = func(*args, **kwargs)
        return val
    wrapper.num = 0 # Когда создается функция wrapper внутри декоратора, мы добавляем к ней атрибут num, инициализируя его значением 0
    return wrapper

@counter
def greet(name):
    return f'Hello {name}!'

print(greet('Timur'))
print(greet('Ruslan'))
print(greet('Arthur'))
print(greet('Gvido'))

Вызов greet: 1
Hello Timur!
Вызов greet: 2
Hello Ruslan!
Вызов greet: 3
Hello Arthur!
Вызов greet: 4
Hello Gvido!


Поскольку атрибут num сохраняется в объекте wrapper, он поддерживает состояние между вызовами. Это означает, что каждый вызов функции, декорированной с помощью counter, будет увеличивать этот счетчик, даже если сами вызовы происходят в разных местах или при разных аргументах.

Атрибут num можно добавлять к функции после её определения.

Предположим, если бы вы попытались инициализировать num до функции wrapper, это привело бы к ошибке, так как вы не можете присваивать атрибут функции, которая ещё не существует в момент выполнения

в Python атрибуты можно создавать и изменять через точечную нотацию, и это работает как для объектов, так и для функций. Атрибуты объекта (включая функции) можно добавить, обновить или удалить с помощью точки.

Когда мы говорим о создании атрибутов у функции, мы используем точечную нотацию, чтобы динамически добавлять атрибуты к функции, поскольку функции в Python — это объекты первого класса, которые могут иметь свои атрибуты.

Атрибуты добавляются и обновляются с использованием точечной нотации, т.е. wrapper.num = 0. Если атрибут не существует, Python создаст его; если существует — просто обновит значение.

Декоратор замедления времени выполнения функции

Иногда полезно иметь декоратор, который замедляет время выполнения функции. Создадим декоратор slow_down, который будет добавлять задержку выполнения программы в 1 секунду, прежде чем вызовет декорируемую функцию.

In [4]:
import functools
import time

def slow_down(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper

@slow_down
def countdown(number):
    if number < 1:
        print('Конец!')
    else:
        print(number)
        countdown(number - 1)   # Рекурсивный вызов
        
countdown(5)

5
4
3
2
1
Конец!


Декоратор square
Реализуйте декоратор square, который возводит возвращаемое значение декорируемой функции во вторую степень. 

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Гарантируется, что возвращаемым значением декорируемой функции является объект типа int или float.

Примечание 2. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 3. В тестирующую систему сдайте программу, содержащую только необходимый декоратор square, но не код, вызывающий его.

In [2]:
import functools

def square(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) ** 2
    return wrapper

@square
def add(a, b):
    return a + b

print(add(3, 7))

@square
def add(a, b):
    '''прекрасная функция'''
    return a + b

print(add(1, 1))
print(add.__name__)
print(add.__doc__)

100
4
add
прекрасная функция


Декоратор returns_string
Реализуйте декоратор returns_string, который проверяет, что возвращаемое значение декорируемой функции принадлежит типу str. Если возвращаемое значение принадлежит какому-либо другому типу, декоратор должен возбуждать исключение TypeError.

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимый декоратор returns_string, но не код, вызывающий его.

In [4]:
import functools

def returns_string(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        if isinstance(result, str):
            return result
        else:
            raise TypeError
    return wrapper

@returns_string
def beegeek():
    return 'beegeek'
    
print(beegeek())

@returns_string
def add(a, b):
    return a + b

try:
    print(add(3, 7))
except TypeError as e:
    print(type(e))

beegeek
<class 'TypeError'>


Декоратор trace
Реализуйте декоратор trace, который выводит отладочную информацию о декорируемой функции во время ее выполнения, а именно: имя функции, переданные аргументы и возвращаемое значение в следующем формате:

TRACE: вызов <имя функции>() с аргументами: <кортеж позиционных аргументов>, <словарь именованных аргументов>
TRACE: возвращаемое значение <имя функции>(): <возвращаемое значение>
Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимый декоратор trace, но не код, вызывающий его.

In [18]:
import functools

def trace(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f'TRACE: вызов {func.__name__}() с аргументами: {args}, {kwargs}')
        result = func(*args, **kwargs)
        print(f"TRACE: возвращаемое значение {func.__name__}(): {f"'{result}'" if isinstance(result, str) else result}")
        return result
    return wrapper

@trace
def say(name, line):
    return f'{name}: {line}'
    
say('Jane', 'Hello, World')

@trace
def sub(a, b, c):
    '''прекрасная функция'''
    return a - b + c
    
print(sub.__name__)
print(sub.__doc__)
sub(20, 5, c=10)

@trace
def beegeek():
    '''beegeek docs'''
    return 'beegeek'

print(beegeek())    
print(beegeek.__name__)
print(beegeek.__doc__)

TRACE: вызов say() с аргументами: ('Jane', 'Hello, World'), {}
TRACE: возвращаемое значение say(): 'Jane: Hello, World'
sub
прекрасная функция
TRACE: вызов sub() с аргументами: (20, 5), {'c': 10}
TRACE: возвращаемое значение sub(): 25
TRACE: вызов beegeek() с аргументами: (), {}
TRACE: возвращаемое значение beegeek(): 'beegeek'
beegeek
beegeek
beegeek docs


In [ ]:
import functools

def trace(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f'TRACE: вызов {func.__name__}() с аргументами: {args}, {kwargs}')
        f = func(*args, **kwargs)
        print(f'TRACE: возращаемое значение {func.__name__}(): {repr(f)}')
        return f
    return wrapper

Декораторы с аргументами

Назовем рассматриваемые до сих пор декораторы стандартными. Стандартный декоратор — это функция, которая принимает в качестве аргумента функцию и возвращает другую функцию, подменяющую исходную.

Рассмотрим определение трех декораторов stars, lines и equals:

In [19]:
import functools

def stars(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('*' * 30)
        return func(*args, **kwargs)
    return wrapper

def lines(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('-' * 10)
        return func(*args, **kwargs)
    return wrapper

def equals(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('=' * 40)
        return func(*args, **kwargs)
    return wrapper

In [20]:
@stars
def add(a, b):
    return a + b

@lines
def mult(a, b):
    return a * b

@equals
def diff(a, b):
    return a - b

print(add(3, 9))
print(mult(10, 20))
print(diff(100, 1))

******************************
12
----------
200
99


Несложно заметить, что все три декоратора выполняют примерно одно и то же. Куда удобнее иметь один декоратор, который можно настраивать, передавая ему соответствующие аргументы. Для того чтобы создать декоратор, принимающий аргументы, необходимо добавить еще один уровень вложенности, то есть создать функцию, которая возвращает нужный декоратор:

In [22]:
def print_symbols(symbol, length):
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(symbol * length)
            return func(*args, **kwargs)
        return wrapper
    return decorator

In [23]:
@print_symbols('*', 30)
def add(a, b):
    return a + b

@print_symbols('-', 10)
def mult(a, b):
    return a * b

@print_symbols('=', 40)
def diff(a, b):
    return a - b

print(add(3, 9))
print(mult(10, 20))
print(diff(100, 1))

******************************
12
----------
200
99


Функция print_symbols() на первый взгляд может показаться декоратором, но на самом деле таковым не является. Это обычная функция, которая принимает аргументы symbol и length, а затем возвращает декоратор. В свою очередь, он декорирует функции add(), mult() и diff()

Нужно помнить, что декоратором является функция, которая принимает функцию в качестве аргумента и возвращает функцию. В нашем примере функция print_symbols() не удовлетворяет этому условию, так как она не принимает функцию в качестве аргумента. В то время как функция decorator(), которая возвращает функцию wrapper(), является декоратором.

Несмотря на то что функция print_symbols() декоратором как таковым не является, мы все равно называем ее декоратором с аргументами.

Рассмотрим еще несколько примеров декораторов с аргументами.

Декоратор delayed

Реализуем декоратор delayed, который создает требуемую задержку выполнения кода. Такое поведение иногда требуется для мониторинга доступности какого-нибудь ресурса.

In [24]:
import functools
import time

def delayed(delay=2):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(f'Спим {delay} сек.')
            time.sleep(delay)
            value = func(*args, **kwargs)
            return value
        return wrapper
    return decorator

Задекорируем данным декоратором рекурсивную функцию обратного отсчета.

In [25]:
@delayed(1)
def countdown(number):
    if number < 1:
        print('Конец!')
    else:
        print(number)
        countdown(number - 1)
        
countdown(5)

Спим 1 сек.
5
Спим 1 сек.
4
Спим 1 сек.
3
Спим 1 сек.
2
Спим 1 сек.
1
Спим 1 сек.
Конец!


Функция delayed() не является декоратором. Это обычная функция, которая принимает аргумент delay, а затем возвращает декоратор. В свою очередь, он декорирует функцию countdown()

Декоратор timer

Рассмотрим декоратор timer, который подсчитывает время выполнения функции. Для более точного подсчета декоратор принимает аргумент iters, который задает количество измерений.

In [26]:
import functools, time

def timer(iters=1):
    def decorator(func):   
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            total = 0
            for i in range(iters):
                start = time.perf_counter()
                value = func(*args, **kwargs)
                end = time.perf_counter()
                total += end - start
            print(f'Среднее время выполнения {func.__name__}: {round(total/iters, 4)} сек.')
            return value
        return wrapper
    return decorator

In [27]:
@timer(iters=1000)
def test(n):
    return sum([(i/99)**2 for i in range(n)])

@timer(iters=3)
def sleep(n):
    time.sleep(n)

res1 = test(10000)
res2 = sleep(4)

print(f'Результат функции test = {res1}')
print(f'Результат функции sleep = {res2}')

Среднее время выполнения test: 0.0015 сек.
Среднее время выполнения sleep: 4.0003 сек.
Результат функции test = 34005033.67003367
Результат функции sleep = None


Функция timer() не является декоратором. Это обычная функция, которая принимает аргумент iters, а затем возвращает декоратор. В свою очередь, он декорирует функции test() и sleep()

Декоратор repeater

Рассмотрим декоратор repeater, который вызывает декорируемую функцию переданное в качестве аргумента количество раз.

In [28]:
import functools

def repeater(repeat=1):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(1, repeat + 1):
                print(f'{i}-й запуск функции.')
                value = func(*args, **kwargs)
            return value
        return wrapper
    return decorator

In [29]:
@repeater(repeat=5)
def beegeek():
    print('beegeek')

beegeek()

1-й запуск функции.
beegeek
2-й запуск функции.
beegeek
3-й запуск функции.
beegeek
4-й запуск функции.
beegeek
5-й запуск функции.
beegeek


Несмотря на то что декоратор repeater имеет значение по умолчанию для аргумента repeat, применить его как стандартный декоратор, не указав скобки, мы не можем. 

In [30]:
@repeater
def beegeek():
    print('beegeek')

beegeek()

TypeError: repeater.<locals>.decorator() missing 1 required positional argument: 'func'

Правильная версия кода имеет вид:

In [31]:
@repeater()                       # используется значение по умолчанию repeat=1
def beegeek():
    print('beegeek')

beegeek()

1-й запуск функции.
beegeek


Примечания

Примечание 1. Напомним, что к одной функции можно применять несколько декораторов. При использовании этой возможности каждый декоратор должен указываться на отдельной строке. Несколько декораторов, следующих подряд, не декорируют друг друга. Они изменяют функцию и ее измененный вариант в качестве аргумента передают друг другу в порядке следования.

Возьмем декораторы repeater и delayed

In [1]:
import functools
import time

def repeater(repeat=1):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(1, repeat + 1):
                print(f'{i}-й запуск функции.')
                value = func(*args, **kwargs)
            return value
        return wrapper
    return decorator

def delayed(delay=2):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(f'Спим {delay} сек.')
            time.sleep(delay)
            value = func(*args, **kwargs)
            return value
        return wrapper
    return decorator

и применим их к функции monitor(), которая имитирует проверку сайта на доступность

In [2]:
@repeater(repeat=5)
@delayed(delay=1)
def monitor(url):
    print(f'Проверка {url} на доступность.')
    
monitor('https://stepik.org/')

# Два декоратора сверху эквивалетны записи ниже
monitor = repeater(repeat=5)(delayed(delay=1)(monitor))

1-й запуск функции.
Спим 1 сек.
Проверка https://stepik.org/ на доступность.
2-й запуск функции.
Спим 1 сек.
Проверка https://stepik.org/ на доступность.
3-й запуск функции.
Спим 1 сек.
Проверка https://stepik.org/ на доступность.
4-й запуск функции.
Спим 1 сек.
Проверка https://stepik.org/ на доступность.
5-й запуск функции.
Спим 1 сек.
Проверка https://stepik.org/ на доступность.


Обратите внимание, в какой последовательности выполняются декораторы. Сначала выполняется декоратор delayed – происходит задержка выполнения на 1 секунду, а затем выполняется декоратор repeater, который повторяет предыдущую операцию 5 раз.

Если изменить очередность декораторов в коде, то логика выполнения поменяется.

In [3]:
@delayed(delay=1)
@repeater(repeat=5)
def monitor(url):
    print(f'Проверка {url} на доступность.')
    
monitor('https://stepik.org/')

# Два декоратора сверху эквивалетны записи ниже
monitor = delayed(delay=1)(repeater(repeat=5)(monitor))

Спим 1 сек.
1-й запуск функции.
Проверка https://stepik.org/ на доступность.
2-й запуск функции.
Проверка https://stepik.org/ на доступность.
3-й запуск функции.
Проверка https://stepik.org/ на доступность.
4-й запуск функции.
Проверка https://stepik.org/ на доступность.
5-й запуск функции.
Проверка https://stepik.org/ на доступность.


Когда мы применяем декораторы к функции, они действительно применяются снизу вверх, то есть сначала самый нижний оборачивает исходную функцию, затем следующий оборачивает функцию, обернутую первым декоратором, затем следующий оборачивает уже эту обертку, и так до самого верхнего декоратора.

В этом можно убедиться, запустив следующий код:

In [10]:
def decorator1(func):
    print("Выполняется декоратор 1")

    def inner(*args, **kwargs):
        print('Обертка 1')
        return func(*args, **kwargs)

    return inner


def decorator2(func):
    print("Выполняется декоратор 2")

    def inner(*args, **kwargs):
        print('Обертка 2')
        return func(*args, **kwargs)

    return inner


@decorator2
@decorator1
def func():
    print("Выполняется функция")

Выполняется декоратор 1
Выполняется декоратор 2


А вот когда мы вызываем функцию, обернутую несколькими декораторами, то работа выполняется уже наоборот, сверху вниз. Сначала выполняется функция внутри самого верхнего декоратора, она вызывает функцию, которую обернула до этого, то есть вложенную функцию следующего декоратора. Она в свою очередь вызывает функцию, которую обернула до этого, то есть вложенную функцию очередного декоратора по порядку, и так до тех пор, пока не будет выполнена исходная функция.

В этом можно убедиться, запустив следующий код:

In [7]:
def decorator1(func):
    print("Выполняется декоратор 1")

    def inner(*args, **kwargs):
        print('Обертка 1')
        return func(*args, **kwargs)

    return inner


def decorator2(func):
    print("Выполняется декоратор 2")

    def inner(*args, **kwargs):
        print('Обертка 2')
        return func(*args, **kwargs)

    return inner


@decorator2
@decorator1
def func():
    print("Выполняется функция")

func()

Выполняется декоратор 1
Выполняется декоратор 2
Обертка 2
Обертка 1
Выполняется функция


Декоратор prefix
Реализуйте декоратор prefix, который принимает два аргумента в следующем порядке:

string — произвольная строка
to_the_end — булево значение, по умолчанию равное False
Декоратор должен добавлять строку string к возвращаемому значению декорируемой функции. Если to_the_end имеет значение True, строка string добавляется в конец, если False — в начало.

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Гарантируется, что возвращаемым значением декорируемой функции является объект типа str.

Примечание 2. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 3. В тестирующую систему сдайте программу, содержащую только необходимый декоратор prefix, но не код, вызывающий его.

In [12]:
import functools

def prefix(string: str, to_the_end = False):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            value = func(*args, **kwargs)
            if to_the_end:
                return value + string   
            else:
                return string + value
        return wrapper
    return decorator

@prefix('€')
def get_bonus():
    return '2000'
    
print(get_bonus())

@prefix('$$$', to_the_end=True)
def get_bonus():
    return '2000'
       
print(get_bonus())

€2000
2000$$$


Декоратор make_html
Тег — элемент языка разметки, используемый для форматирования текста. Например, текст, заключённый между начальным тегом <small> и конечным тегом </small>, отображается с меньшим размером, чем основной текст, а текст между тегами <big> и </big> отображается с большим размером.

Реализуйте декоратор make_html(), который принимает один аргумент:

tag — HTML-тег, например, del
Декоратор должен обрамлять возвращаемое значение декорируемой функции в HTML-тег tag.

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Гарантируется, что возвращаемым значением декорируемой функции является объект типа str.

Примечание 2. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 3. В тестирующую систему сдайте программу, содержащую только необходимый декоратор make_html, но не код, вызывающий его

In [14]:
import functools

def make_html(tag: str):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            value = func(*args, **kwargs)
            return f'<{tag}>' + value + f'</{tag}>'
        return wrapper
    return decorator

@make_html('del')
def get_text(text):
    return text
    
print(get_text('Python'))

@make_html('i')
@make_html('del')
def get_text(text):
    return text
    
print(get_text(text='decorators are so cool!'))

<del>Python</del>
<i><del>decorators are so cool!</del></i>


Декоратор repeat
Реализуйте декоратор repeat, который принимает один аргумент:

times — натуральное число
Декоратор должен вызывать декорируемую функцию times раз.

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимый декоратор repeat, но не код, вызывающий его.

In [17]:
import functools

def repeat(times: int):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(times):
                value = func(*args, **kwargs)
            return value
        return wrapper
    return decorator

@repeat(3)
def say_beegeek():
    '''documentation'''
    print('beegeek')
    
say_beegeek()

@repeat(4)
def say_beegeek():
    '''documentation'''
    print('beegeek')
    
print(say_beegeek.__name__)
print(say_beegeek.__doc__)

beegeek
beegeek
beegeek
say_beegeek
documentation


Декоратор strip_range
Реализуйте декоратор strip_range, который принимает три аргумента в следующем порядке:

start — неотрицательное целое число
end — неотрицательное целое число
char — одиночный символ, по умолчанию равный точке .
Декоратор должен изменять возвращаемое значение декорируемой функции, заменяя все символы в диапазоне индексов от start (включительно) до end (не включительно) на символ char.

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Гарантируется, что возвращаемым значением декорируемой функции является объект типа str.

Примечание 2. Гарантируется, что start < end.

Примечание 3. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 4. В тестирующую систему сдайте программу, содержащую только необходимый декоратор strip_range, но не код, вызывающий его.

In [20]:
import functools

def strip_range(start: int, end: int, char='.'):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            value = func(*args, **kwargs)
            length = len(value)
            new_start = min(start, length)
            new_end = min(end, length)
            return value[:new_start] + char * (new_end - new_start) + value[new_end:]
        return wrapper
    return decorator

@strip_range(3, 5)
def beegeek():
    return 'beegeek'
    
print(beegeek())

@strip_range(3, 20, '_')
def beegeek():
    return 'beegeek'
    
print(beegeek())

@strip_range(20, 30)
def beegeek():
    return 'beegeek'
    
print(beegeek())

bee..ek
bee____
beegeek


Декоратор returns 🌶️
Реализуйте декоратор returns, который принимает один аргумент:

datatype — тип данных
Декоратор должен проверять, что возвращаемое значение декорируемой функции принадлежит типу datatype. Если возвращаемое значение принадлежит какому-либо другому типу, декоратор должен возбуждать исключение TypeError.

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимый декоратор returns, но не код, вызывающий его.

In [24]:
import functools

def returns(datatype):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            value = func(*args, **kwargs)
            if isinstance(value, datatype):
                return value
            else:
                raise TypeError
        return wrapper
    return decorator

@returns(int)
def add(a, b):
    return a + b

print(add(10, 5))

@returns(int)
def add(a, b):
    return a + b

try:
    print(add('199', '1'))
except TypeError as e:
    print(type(e))
    
@returns(list)
def beegeek():
    '''beegeek docs'''
    return 'beegeek'

print(beegeek.__name__)
print(beegeek.__doc__)

try:
    print(beegeek())
except TypeError as e:
    print(type(e))
    
@returns(list)
def append_this(li, elem):
    '''append_this docs'''
    return li + [elem]

print(append_this.__name__)
print(append_this.__doc__)
print(append_this([1, 2, 3], elem=4))

15
<class 'TypeError'>
beegeek
beegeek docs
<class 'TypeError'>
append_this
append_this docs
[1, 2, 3, 4]


Декоратор takes 🌶️
Реализуйте декоратор takes, который принимает произвольное количество позиционных аргументов, каждый из которых является типом данных.

Декоратор должен проверять, что аргументы, передаваемые в декорируемую функцию, принадлежат одному из этих типов. Если хотя бы один аргумент не принадлежит одному из данных типов, декоратор должен возбуждать исключение TypeError.

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимый декоратор takes, но не код, вызывающий его.

In [33]:
import functools

def takes(*args_s):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            value = func(*args, **kwargs)
            if not all(isinstance(i, args_s) for i in args):
                return TypeError
            if not all(isinstance(i, args_s) for i in kwargs.values()):
                return TypeError
            return value                
        return wrapper
    return decorator

# @takes(int, str)
# def repeat_string(string, times):
#     return string * times
# 
# print(repeat_string('bee', 3))
# 
# @takes(list, bool, float, int)
# def repeat_string(string, times):
#     return string * times
# 
# try:
#     print(repeat_string('bee', 4))
# except TypeError as e:
#     print(type(e))
    
# @takes(list)
# def append_this(li, elem):
#     '''append_this docs'''
#     return li + [elem]
# 
# print(append_this.__name__)
# print(append_this.__doc__)
# 
# try:
#     print(append_this([1, 2], 3))
# except TypeError as e:
#     print(type(e))
    
@takes(str)
def beegeek(word, repeat):
    return word * repeat
    
try:
    print(beegeek('beegeek', repeat=2))
except TypeError as e:
    print(type(e))

<class 'TypeError'>


Декоратор add_attrs 🌶️
Реализуйте декоратор add_attrs, который принимает произвольное количество именованных аргументов и устанавливает их в качестве атрибутов декорируемой функции. Названием атрибута должно являться имя аргумента, значением атрибута — значение аргумента.

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Вспомните про атрибут функции __dict__.

Примечание 2. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 3. В тестирующую систему сдайте программу, содержащую только необходимый декоратор add_attrs, но не код, вызывающий его.

In [37]:
import functools

def add_attrs(**kwargs_s):
    def decorator(func):
        for k, v in kwargs_s.items():
            func.__dict__[k] = v
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            value = func(*args, **kwargs)            
            return value                
        return wrapper
    return decorator

@add_attrs(attr1='bee', attr2='geek')
def beegeek():
    return 'beegeek'
    
print(beegeek.attr1)
print(beegeek.attr2)

bee
geek


Не требуется делать ровным счётом ничего при вызове декорируемой функции, значит и декорировать её нет смысла – пусть остаётся как есть

In [ ]:
def add_attrs(**kwargs):
    return lambda fun: [fun.__dict__.update(kwargs), fun][1]

Декоратор ignore_exception 🌶️
Реализуйте декоратор ignore_exception, который принимает произвольное количество позиционных аргументов — типов исключений, и выводит текст:

Исключение <тип исключения> обработано
если во время выполнения декорируемой функции было возбуждено исключение, принадлежащее одному из переданных типов.

Если возбужденное исключение не принадлежит ни одному из переданных типов, оно должно быть возбуждено снова.

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов.

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимый декоратор ignore_exception, но не код, вызывающий его.

In [20]:
import functools

def ignore_exception(*args_s):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            try:
                value = func(*args, **kwargs)
                return value
            except Exception as e:
                if any(isinstance(e, exc) for exc in args_s):
                    print(f'Исключение {type(e).__name__} обработано')
                    return None  # Возвращаем None, чтобы избежать повторного выброса
                else:
                    raise e
        return wrapper
    return decorator


# @ignore_exception(ZeroDivisionError, TypeError, ValueError)
# def f(x):
#     return 1 / x
# 
# f(0)

min = ignore_exception(ZeroDivisionError)(min)

try:
    print(min(1, '2', 3, [4, 5]))
except Exception as e:
    print(type(e))

None


In [ ]:
import functools

def ignore_exception(*exceptions):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except exceptions as e:
                print(f"Исключение {type(e).__name__} обработано")
        return wrapper
    return decorator

Декоратор retry 🌶️
Реализуйте декоратор retry, который принимает один аргумент:

times — натуральное число
Декоратор должен выполнять повторную попытку вызова декорируемой функции, если во время ее выполнения возникает ошибка. Декоратор должен вызывать ее до тех пор, пока не исчерпает количество попыток times, после чего должен возбуждать исключение MaxRetriesException.

Также декоратор должен сохранять имя и строку документации декорируемой функции.

Примечание 1. Не забывайте про то, что декоратор не должен поглощать возвращаемое значение декорируемой функции, а также должен уметь декорировать функции с произвольным количеством позиционных и именованных аргументов. 

Примечание 2. В тестирующую систему сдайте программу, содержащую только необходимый декоратор retry, но не код, вызывающий его. 

In [19]:
class MaxRetriesException(Exception):
    pass

import functools

def retry(times: int):
    def decorator(func):        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(times):
                try:
                    return func(*args, **kwargs)
                except:
                    pass
            raise MaxRetriesException
        return wrapper
    return decorator

@retry(3)
def no_way():
    raise ValueError

try:
    no_way()
except Exception as e:
    print(type(e))
    
@retry(8)
def beegeek():
    beegeek.calls = beegeek.__dict__.get('calls', 0) + 1
    if beegeek.calls < 5:
        raise ValueError
    print('beegeek')
    
beegeek()

<class '__main__.MaxRetriesException'>
beegeek


In [ ]:
import functools


class MaxRetriesException(Exception):
    pass


def retry(times):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if times == 0:
                raise MaxRetriesException
            else:
                try:
                    return func(*args, **kwargs)
                except:
                    return retry(times-1)(func)(*args, **kwargs)

        return wrapper

    return decorator